# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=0&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [3]:
# headers definition
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}


In [4]:
# Your code
response = requests.get(url,headers=header)
results = response.json()
results

{'total_count': 20242,
 'pagination': {'page_count': 241, 'current_page': 1, 'per_page': 84},
 'sort': 'sale',
 'articles': [{'sku': 'LT124A00W-C11',
   'name': 'CAYLE - Jean slim - sound grey wash',
   'price': {'original': '49,95\xa0€',
    'promotional': '35,00\xa0€',
    'has_different_prices': False,
    'has_different_original_prices': False,
    'has_different_promotional_prices': False,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['6a', '8a', '12a', '14a', '16a'],
   'url_key': 'ltb-cayle-jeans-skinny-sound-grey-wash-lt124a00w-c11',
   'media': [{'path': 'LT/12/4A/00/WC/11/LT124A00W-C11@2.jpg',
     'role': 'DEFAULT',
     'packet_shot': True}],
   'brand_name': 'LTB',
   'is_premium': False,
   'family_articles': [{'sku': 'LT124A00W-C11',
     'url_key': 'ltb-cayle-jeans-skinny-sound-grey-wash-lt124a00w-c11',
     'media': [{'path': 'LT/12/4A/00/WC/11/LT124A00W-C11@2.jpg',
       'role': 'FAMILY',
       'packet_shot': True}],
     'name': 'CAYLE - Jean slim

#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [5]:
flattened_data = json_normalize(results)
flattened_data
flattened_data['articles']
flattened_data.articles[0]
articles = json_normalize(flattened_data.articles[0])   
articles.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
0,NaN,LTB,"[{'sku': 'LT124A00W-C11', 'url_key': 'ltb-cayl...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4A/00/WC/11/LT124A00W-C11@2.j...,CAYLE - Jean slim - sound grey wash,False,False,False,False,"49,95 €","35,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",LT124A00W-C11,https://ccp-et.metrigo.zalan.do/event/sbv?z=19...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=1...,ccp,ltb-cayle-jeans-skinny-sound-grey-wash-lt124a0...
1,NaN,LTB,"[{'sku': 'LT124C00E-K16', 'url_key': 'ltb-lanc...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4C/00/EK/16/LT124C00E-K16@8.j...,LANCE - Short en jean - lazaro wash,False,False,False,False,"44,95 €","36,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",LT124C00E-K16,https://ccp-et.metrigo.zalan.do/event/sbv?z=19...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=1...,ccp,ltb-lance-short-en-jean-lt124c00e-k16
2,NaN,LTB,"[{'sku': 'LT123C007-K15', 'url_key': 'ltb-judi...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/3C/00/7K/15/LT123C007-K15@2.j...,JUDIE - Short en jean - senate wash,False,False,False,False,"39,95 €","32,00 €",clothing,"[6a, 12a, 14a, 16a]",LT123C007-K15,https://ccp-et.metrigo.zalan.do/event/sbv?z=19...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=1...,ccp,ltb-judie-short-en-jean-lt123c007-k15
3,NaN,Nike Sportswear,"[{'sku': 'NI113D07F-A11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -70...",False,[{'path': 'NI/11/3D/07/FA/11/NI113D07F-A11@3.j...,AIR MAX 97 - Baskets basses - white,False,True,True,False,"139,95 €","42,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5]",NI113D07F-A11,NaN,NaN,NaN,nike-sportswear-air-max-97-baskets-basses-whit...
4,NaN,Nike Sportswear,"[{'sku': 'NI114D0AF-Q11', 'url_key': 'nike-spo...","[{'key': 'campaign', 'value': 'HOT DROP', 'tra...",False,[{'path': 'NI/11/4D/0A/FQ/11/NI114D0AF-Q11@8.j...,REACT PRESTO - Baskets basses - black/white,False,True,True,False,"84,95 €","34,00 €",shoe,"[28, 28.5, 29.5, 30, 33.5, 34, 35]",NI114D0AF-Q11,NaN,NaN,NaN,nike-sportswear-react-presto-baskets-basses-bl...


In [6]:
# Get the total number of pages
pages = results['pagination']['page_count']

# Your code
goods = pd.DataFrame()
for i in range(pages):
    k = i*84
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset={k}&sort=sale'
    response = requests.get(url,headers=header)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data2 = json_normalize(flattened_data.articles[0])
    art = flattened_data2.set_index('sku')
    goods = goods.append(art)


/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
goods.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
sku,,,,,,,,,,,,,,,,,,,
LT124A00W-C11,NaN,LTB,"[{'sku': 'LT124A00W-C11', 'url_key': 'ltb-cayl...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4A/00/WC/11/LT124A00W-C11@2.j...,CAYLE - Jean slim - sound grey wash,False,False,False,False,"49,95 €","35,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-cayle-jeans-skinny-sound-grey-wash-lt124a0...
LT124C00E-K16,NaN,LTB,"[{'sku': 'LT124C00E-K16', 'url_key': 'ltb-lanc...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4C/00/EK/16/LT124C00E-K16@8.j...,LANCE - Short en jean - lazaro wash,False,False,False,False,"44,95 €","36,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-lance-short-en-jean-lt124c00e-k16
LT123C007-K15,NaN,LTB,"[{'sku': 'LT123C007-K15', 'url_key': 'ltb-judi...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/3C/00/7K/15/LT123C007-K15@2.j...,JUDIE - Short en jean - senate wash,False,False,False,False,"39,95 €","32,00 €",clothing,"[6a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-judie-short-en-jean-lt123c007-k15
NI113D07F-A11,NaN,Nike Sportswear,"[{'sku': 'NI113D07F-A11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -70...",False,[{'path': 'NI/11/3D/07/FA/11/NI113D07F-A11@3.j...,AIR MAX 97 - Baskets basses - white,False,True,True,False,"139,95 €","42,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5]",NaN,NaN,NaN,nike-sportswear-air-max-97-baskets-basses-whit...
NI114D0AF-Q11,NaN,Nike Sportswear,"[{'sku': 'NI114D0AF-Q11', 'url_key': 'nike-spo...","[{'key': 'campaign', 'value': 'HOT DROP', 'tra...",False,[{'path': 'NI/11/4D/0A/FQ/11/NI114D0AF-Q11@8.j...,REACT PRESTO - Baskets basses - black/white,False,True,True,False,"84,95 €","34,00 €",shoe,"[28, 28.5, 29.5, 30, 33.5, 34, 35]",NaN,NaN,NaN,nike-sportswear-react-presto-baskets-basses-bl...


#### Display the trending brand in DataFrame

In [8]:
# your code
brands = pd.DataFrame(goods['brand_name'].value_counts())
brands.head()

,brand_name
Friboo,855
OVS,792
Name it,660
GAP,602
Benetton,581


#### Display the brand with maximal total discount (sum of discounts on all goods)

In [9]:
#Our data is still text. Convert prices into numbers.
# your code

goods[['price.original_num', 'price.promotional_num']] = goods[['price.original','price.promotional']].replace('[\€,]','', regex=True).astype(int).div(100)


In [10]:
goods.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,...,price.original,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key,price.original_num,price.promotional_num
sku,,,,,,,,,,,,,,,,,,,,,
LT124A00W-C11,NaN,LTB,"[{'sku': 'LT124A00W-C11', 'url_key': 'ltb-cayl...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4A/00/WC/11/LT124A00W-C11@2.j...,CAYLE - Jean slim - sound grey wash,False,False,False,...,"49,95 €","35,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-cayle-jeans-skinny-sound-grey-wash-lt124a0...,49.95,35.0
LT124C00E-K16,NaN,LTB,"[{'sku': 'LT124C00E-K16', 'url_key': 'ltb-lanc...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4C/00/EK/16/LT124C00E-K16@8.j...,LANCE - Short en jean - lazaro wash,False,False,False,...,"44,95 €","36,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-lance-short-en-jean-lt124c00e-k16,44.95,36.0
LT123C007-K15,NaN,LTB,"[{'sku': 'LT123C007-K15', 'url_key': 'ltb-judi...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/3C/00/7K/15/LT123C007-K15@2.j...,JUDIE - Short en jean - senate wash,False,False,False,...,"39,95 €","32,00 €",clothing,"[6a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-judie-short-en-jean-lt123c007-k15,39.95,32.0
NI113D07F-A11,NaN,Nike Sportswear,"[{'sku': 'NI113D07F-A11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -70...",False,[{'path': 'NI/11/3D/07/FA/11/NI113D07F-A11@3.j...,AIR MAX 97 - Baskets basses - white,False,True,True,...,"139,95 €","42,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5]",NaN,NaN,NaN,nike-sportswear-air-max-97-baskets-basses-whit...,139.95,42.0
NI114D0AF-Q11,NaN,Nike Sportswear,"[{'sku': 'NI114D0AF-Q11', 'url_key': 'nike-spo...","[{'key': 'campaign', 'value': 'HOT DROP', 'tra...",False,[{'path': 'NI/11/4D/0A/FQ/11/NI114D0AF-Q11@8.j...,REACT PRESTO - Baskets basses - black/white,False,True,True,...,"84,95 €","34,00 €",shoe,"[28, 28.5, 29.5, 30, 33.5, 34, 35]",NaN,NaN,NaN,nike-sportswear-react-presto-baskets-basses-bl...,84.95,34.0


In [11]:
goods['total_discount'] =  goods['price.original_num'] - goods['price.promotional_num']

In [12]:
goods.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,...,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key,price.original_num,price.promotional_num,total_discount
sku,,,,,,,,,,,,,,,,,,,,,
LT124A00W-C11,NaN,LTB,"[{'sku': 'LT124A00W-C11', 'url_key': 'ltb-cayl...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4A/00/WC/11/LT124A00W-C11@2.j...,CAYLE - Jean slim - sound grey wash,False,False,False,...,"35,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-cayle-jeans-skinny-sound-grey-wash-lt124a0...,49.95,35.0,14.95
LT124C00E-K16,NaN,LTB,"[{'sku': 'LT124C00E-K16', 'url_key': 'ltb-lanc...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/4C/00/EK/16/LT124C00E-K16@8.j...,LANCE - Short en jean - lazaro wash,False,False,False,...,"36,00 €",clothing,"[6a, 8a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-lance-short-en-jean-lt124c00e-k16,44.95,36.0,8.95
LT123C007-K15,NaN,LTB,"[{'sku': 'LT123C007-K15', 'url_key': 'ltb-judi...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'LT/12/3C/00/7K/15/LT123C007-K15@2.j...,JUDIE - Short en jean - senate wash,False,False,False,...,"32,00 €",clothing,"[6a, 12a, 14a, 16a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=52...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=5...,ccp,ltb-judie-short-en-jean-lt123c007-k15,39.95,32.0,7.95
NI113D07F-A11,NaN,Nike Sportswear,"[{'sku': 'NI113D07F-A11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -70...",False,[{'path': 'NI/11/3D/07/FA/11/NI113D07F-A11@3.j...,AIR MAX 97 - Baskets basses - white,False,True,True,...,"42,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5]",NaN,NaN,NaN,nike-sportswear-air-max-97-baskets-basses-whit...,139.95,42.0,97.95
NI114D0AF-Q11,NaN,Nike Sportswear,"[{'sku': 'NI114D0AF-Q11', 'url_key': 'nike-spo...","[{'key': 'campaign', 'value': 'HOT DROP', 'tra...",False,[{'path': 'NI/11/4D/0A/FQ/11/NI114D0AF-Q11@8.j...,REACT PRESTO - Baskets basses - black/white,False,True,True,...,"34,00 €",shoe,"[28, 28.5, 29.5, 30, 33.5, 34, 35]",NaN,NaN,NaN,nike-sportswear-react-presto-baskets-basses-bl...,84.95,34.0,50.95


In [13]:
total_discounts = pd.DataFrame(goods.groupby(['brand_name']).sum()['total_discount']).sort_values(by=['total_discount'], ascending=False)

In [14]:
total_discounts.head()

,total_discount
brand_name,
Friboo,9338.78
Polo Ralph Lauren,8205.17
Naturino,6478.16
J.CREW,5787.53
Tommy Hilfiger,5692.51


#### Display the brands without discount at all

In [15]:
# your code
total_discounts[total_discounts['total_discount']<=5].sort_values(by=['total_discount'], ascending=True)

,total_discount
brand_name,
Teva,0.00
Reusch,4.99


El día del lab no hay brands sin descuentos, muestro las que tienen descuentos menores a 5